In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import scipy
import itertools
from scipy import linalg
import matplotlib as mpl

from sklearn import mixture
import gmr
from gmr.utils import check_random_state
from gmm_lbd import *




In [2]:
from matplotlib.patches import Ellipse
from itertools import cycle
colors = ['r','v','b']
def plot_ellipses(ax,gmm,colors=None,X=None):
    if colors is not None:
        colors = cycle(colors)
    else :
        colors = cycle(['r', 'g', 'b', 'c', 'm'])
    for factor in np.linspace(0.5, 4.0, 8):
        for i, (mean, covar, color) in enumerate(zip(gmm.means_, gmm._get_covars(), color_iter)):
            
            #if not np.any(Y_ == i):
            #    continue
            #plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)
            ## valeur propre, vecteur propre
            v, w = linalg.eigh(covar)
            u = w[0] / linalg.norm(w[0])
            angle = np.arctan(u[1] / u[0])
            angle = 180 * angle / np.pi  # convert to degrees
            width, height = factor * np.sqrt(v)
                   
            ell = Ellipse(xy=mean, width=width, height=height,
                          angle=angle)
            ell.set_alpha(0.25)
            if colors is not None:
                ell.set_color(next(colors))
            ax.add_artist(ell)


In [3]:

# Number of samples per component
n_samples = 800

# Generate random sample following a sine curve
np.random.seed(0)
X = np.zeros((n_samples, 2))

step = 4 * np.pi / n_samples

for i in xrange(X.shape[0]):
    x = i * step - 6
    X[i, 0] = x + np.random.normal(0, 0.1)
    X[i, 1] = 3 * (np.sin(x) + np.random.normal(0, .2))
    
plt.scatter(X[:,0],X[:,1])

In [4]:
# Essay woth multiple dimension
n_samples = 300
np.random.seed(2)
Y = np.zeros((n_samples, 3))
Y [:,0] = np.linspace(1,10,n_samples)
step = 4 * np.pi / n_samples
for i in xrange(Y.shape[0]):   
    Y[i, 1] = np.sin(Y[i,0])
    Y[i, 2] = np.sin(Y[i,0]) + np.random.normal(0, .2)

    
plt.scatter(Y[:,0],Y[:,1])
plt.scatter(Y[:,0],Y[:,2])

In [5]:
manager=GmmManager()
manager.add_dataset(Y)
manager.gen_gmm('0',n_components_range=[3])
gmm = manager.gmms['0']

In [6]:
gmm.means_
gmm.sample(10)

array([[ 1.7894657 ,  0.97749608,  1.0409079 ],
       [ 3.5305713 , -0.22659315, -0.28657256],
       [ 9.49707199, -0.09144012,  0.09407005],
       [ 4.7976711 , -1.39492532, -1.49460752],
       [ 3.95481403, -0.862308  , -0.45215446],
       [ 6.3370747 , -0.09563875, -0.0411858 ],
       [ 7.0601388 ,  0.56645509,  0.65666493],
       [ 4.2838346 , -0.7123486 , -0.35845897],
       [ 9.10671322,  0.25605143,  0.16492098],
       [ 5.49778994, -1.63425202, -1.70644596]])

In [7]:
color_iter = itertools.cycle(['r', 'g', 'b', 'c', 'm'])

title = 'Expectation-maximization'
clf = mixture.GMM(n_components=5, covariance_type='full', n_iter=100)
clf.fit(X)
Y_ = clf.predict(X)
splot = plt.subplot(1,1,1)

for i, (mean, covar, color) in enumerate(zip(
            clf.means_, clf._get_covars(), color_iter)):
        ## valeur propre, vecteur propre
        v, w = linalg.eigh(covar)
        u = w[0] / linalg.norm(w[0])
        # as the DP will not use every component it has access to
        # unless it needs it, we shouldn't plot the redundant
        # components.
        if not np.any(Y_ == i):
            continue
        plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, 2*v[0], v[1], 180 + angle, color=color)
        ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.5)
        splot.add_artist(ell)

plt.xlim(-6, 4 * np.pi - 6)
plt.ylim(-5, 5)
plt.title(title)
plt.xticks(())
plt.yticks(())

 

([], <a list of 0 Text yticklabel objects>)

In [8]:
sample = clf.sample(3000)
plt.scatter(sample[:, 0], sample[:, 1], color='red')
plt.scatter(X[:, 0], X[:, 1],0.3)


In [9]:
datas = SanitizeRecordsForGmm()
datas._X = X
print datas._X

datas.gen_gmm()#gmm_type='custom') 

[[-5.82359477  1.07834082]
 [-5.88641824  2.2279241 ]
 [-5.78182827  0.34194518]
 ..., 
 [ 6.60735983  1.32141881]
 [ 6.44256349  1.59464486]
 [ 6.41261958  0.47134925]]


ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

In [ ]:

plt.figure(figsize=(15, 5))
plot_ellipses(plt.gca(), clf)
plt.scatter(X[:, 0], X[:, 1])

        

In [ ]:
from gmr import GMM, plot_error_ellipses

random_state = check_random_state(0)
gmm = gmr.GMM(n_components=5, random_state=random_state)
gmm.from_samples(X)
plt.figure(figsize=(15, 5))
plot_error_ellipses(plt.gca(), gmm, colors=["r", "g", "b"])
plt.scatter(X[:, 0], X[:, 1])



In [ ]:
sample = gmm.sample(300)
plt.scatter(sample[:, 0], sample[:, 1], color='red')
plt.scatter(X[:, 0], X[:, 1],0.3)



In [ ]:
gmm.to_probability_density(X)
plt.plot(X[:,0],gmm.to_probability_density(X))


In [ ]:
gmm.to